In [1]:
import carla
import math
import random
import time
import os
import numpy as np
import cv2
import open3d as o3d
from matplotlib import cm

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
client = carla.Client('localhost', 2000)
world = client.get_world()  # access everything in carla

bp_lib = world.get_blueprint_library()  # 蓝图库
spawn_points = world.get_map().get_spawn_points()  # 随机起点

In [ ]:
drone_init_trans1 = carla.Transform(carla.Location(z=60, x=-80), carla.Rotation(pitch=-90))


In [ ]:
# 声明vehicle sensor
ego_vehicle_bp = bp_lib.find('vehicle.mercedes-benz.coupe')
ego_vehicle_bp.set_attribute('color', '0, 0, 0')
ego_vehicle = world.try_spawn_actor(ego_vehicle_bp, random.choice(spawn_points))


In [ ]:
# Move spectator to view ego vehicle
spectator = world.get_spectator() 
transform = carla.Transform(vehicle.get_transform().transform(carla.Location(x=-4,z=2.5)),vehicle.get_transform().rotation) 
spectator.set_transform(transform)

In [ ]:
# 声明一堆车，自动行驶
# Add traffic and set in motion with Traffic Manager
for i in range(100): 
    vehicle_bp = random.choice(bp_lib.filter('vehicle')) 
    npc = world.try_spawn_actor(vehicle_bp, random.choice(spawn_points))    
for v in world.get_actors().filter('*vehicle*'): 
    v.set_autopilot(True) 


In [ ]:
# 开启所有传感器，录制视频



In [ ]:
# 保存结果



In [ ]:
spectator = world.get_spectator()
spectator.set_transform(camera_init_trans)

In [10]:
# 上帝视角开启在车后
spectator = world.get_spectator()
transform = carla.Transform(
    ego_vehicle.get_transform().transform(carla.Location(x=-4, z=2.5)),
    ego_vehicle.get_transform().rotation
)
spectator.set_transform(transform)

In [7]:
# all cameras
camera_init_trans = carla.Transform(carla.Location(z=2))

camera_bp = bp_lib.find('sensor.camera.rgb')
camera = world.spawn_actor(camera_bp, camera_init_trans, attach_to=ego_vehicle)

sem_camera_bp = bp_lib.find('sensor.camera.semantic_segmentation')
sem_camera = world.spawn_actor(sem_camera_bp, camera_init_trans, attach_to=ego_vehicle)

dvs_camera_bp = bp_lib.find('sensor.camera.dvs')
dvs_camera = world.spawn_actor(dvs_camera_bp, camera_init_trans, attach_to=ego_vehicle)

depth_camera_bp = bp_lib.find('sensor.camera.depth')
depth_camera = world.spawn_actor(depth_camera_bp, camera_init_trans, attach_to=ego_vehicle)

In [8]:
# 实时视频流
def camera_callback(image, data_dict):
    data_dict['image'] = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))

def sem_callback(image, data_dict):
    image.convert(carla.ColorConverter.CityScapesPalette)
    data_dict['sem_image'] = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))

def depth_callback(image, data_dict):
    image.convert(carla.ColorConverter.LogarithmicDepth)
    data_dict['depth_image'] = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))
    
def dvs_callback(data, data_dict):
    dvs_events = np.frombuffer(data.raw_data, dtype=np.dtype([
        ('x', np.uint16), ('y', np.uint16), ('t', np.int64), ('pol', np.bool)
    ]))
    data_dict['dvs_image'] = np.zeros((data.height, data.width, 4), dtype=np.uint8)
    dvs_img = np.zeros((data.height, data.width, 3), dtype=np.uint8)
    dvs_img[dvs_events[:]['y'], dvs_events[:]['x'], dvs_events[:]['pol']*2] = 255
    data_dict['dvs_image'][:, :, 0:3] = dvs_img

In [9]:
image_w = camera_bp.get_attribute("image_size_x").as_int()
image_h = camera_bp.get_attribute("image_size_y").as_int()

sensor_data = {
    'image': np.zeros((image_h, image_w, 4)),
    'sem_image': np.zeros((image_h, image_w, 4)),
    'depth_image': np.zeros((image_h, image_w, 4)),
    'dvs_image': np.zeros((image_h, image_w, 4))
}

cv2.namedWindow("All Cameras", cv2.WINDOW_AUTOSIZE)
top_row = np.concatenate((sensor_data['image'], sensor_data['sem_image']), axis=1)
bottom_row = np.concatenate((sensor_data['depth_image'], sensor_data['dvs_image']), axis=1)
tiled = np.concatenate((top_row, bottom_row), axis=0)

cv2.imshow("All Cameras", tiled)
cv2.waitKey(1)

camera.listen(lambda image: camera_callback(image, sensor_data))
sem_camera.listen(lambda image: sem_callback(image, sensor_data))
depth_camera.listen(lambda image: depth_callback(image, sensor_data))
dvs_camera.listen(lambda image: dvs_callback(image, sensor_data))

while True:
    top_row = np.concatenate((sensor_data['image'], sensor_data['sem_image']), axis=1)
    bottom_row = np.concatenate((sensor_data['depth_image'], sensor_data['dvs_image']), axis=1)
    tiled = np.concatenate((top_row, bottom_row), axis=0)
    cv2.imshow("All Cameras", tiled)
    if cv2.waitKey(1) == ord('q'):
        break

camera.stop()
sem_camera.stop()
depth_camera.stop()
dvs_camera.stop()

cv2.destroyAllWindows()

QObject::moveToThread: Current thread (0x563f3ed34080) is not the object's thread (0x563f3f287c60).
Cannot move to target thread (0x563f3ed34080)

QObject::moveToThread: Current thread (0x563f3ed34080) is not the object's thread (0x563f3f287c60).
Cannot move to target thread (0x563f3ed34080)

QObject::moveToThread: Current thread (0x563f3ed34080) is not the object's thread (0x563f3f287c60).
Cannot move to target thread (0x563f3ed34080)

QObject::moveToThread: Current thread (0x563f3ed34080) is not the object's thread (0x563f3f287c60).
Cannot move to target thread (0x563f3ed34080)

QObject::moveToThread: Current thread (0x563f3ed34080) is not the object's thread (0x563f3f287c60).
Cannot move to target thread (0x563f3ed34080)

QObject::moveToThread: Current thread (0x563f3ed34080) is not the object's thread (0x563f3f287c60).
Cannot move to target thread (0x563f3ed34080)

QObject::moveToThread: Current thread (0x563f3ed34080) is not the object's thread (0x563f3f287c60).
Cannot move to tar

QObject::moveToThread: Current thread (0x563f3ed34080) is not the object's thread (0x563f3f287c60).
Cannot move to target thread (0x563f3ed34080)

QObject::moveToThread: Current thread (0x563f3ed34080) is not the object's thread (0x563f3f287c60).
Cannot move to target thread (0x563f3ed34080)

/tmp/ipykernel_228788/1547937127.py:15: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  ('x', np.uint16), ('y', np.uint16), ('t', np.int64), ('pol', np.bool)


In [13]:
camera.destroy()
sem_camera.destroy()
depth_camera.destroy()
dvs_camera.destroy()

False

In [18]:
# lidar
lidar_bp = bp_lib.find('sensor.lidar.ray_cast')
lidar_bp.set_attribute('channels', str(64))
lidar_bp.set_attribute('points_per_second', str(90000))
lidar_bp.set_attribute('rotation_frequency', str(40))
lidar_bp.set_attribute('range', str(20))

In [19]:
lidar_location = carla.Location(0, 0, 2)
lidar_rotation = carla.Rotation(0, 0, 0)
lidar_transform = carla.Transform(lidar_location, lidar_rotation)
lidar = world.spawn_actor(lidar_bp, lidar_transform, attach_to=ego_vehicle)

In [21]:
lidar.listen(lambda point_cloud: point_cloud.save_to_disk('point_cloud/%06d.ply'%point_cloud.frame))

In [22]:
lidar.stop()

In [17]:
lidar.destroy()

False